In [ ]:
%pip install sqlalchemy

In [1]:
import pandas as pd
import os
import sys
import psycopg
from psycopg.rows import dict_row
import pandas as pd
from sqlalchemy import create_engine, text

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

DATABASE_URL = os.environ.get('DATABASE_URL')


def connect_and_load():
    try:
        # 2. Create the SQLAlchemy engine
        engine = create_engine(DATABASE_URL)
        
        print("Attempting to connect...")
        
        # 3. Connect to the database
        with engine.connect() as connection:
            print("Successfully connected!")

            # ---------------------------------------------------------
            # STEP A: Check what tables exist in the database
            # ---------------------------------------------------------
            tables_query = text("""
                SELECT table_name 
                FROM information_schema.tables 
                WHERE table_schema = 'public'
            """)
            result = connection.execute(tables_query)
            tables = [row[0] for row in result.fetchall()]
            
            print(f"\nFound tables: {tables}")

            if not tables:
                print("No tables found in the 'public' schema.")
                return

            # ---------------------------------------------------------
            # STEP B: Load data from a specific table (Example)
            # ---------------------------------------------------------
            # Replace 'your_table_name' below with one of the names printed above
            # For now, we default to the first table found in the list.
            target_table = tables[0] 
            
            print(f"\nLoading data from table: '{target_table}'...")
            
            # Using Pandas to read the SQL query directly into a DataFrame
            df = pd.read_sql(f"SELECT * FROM {target_table}", connection)
        

            return df

    except Exception as e:
        print(f"An error occurred: {e}")


data = connect_and_load()
data

Attempting to connect...
Successfully connected!

Found tables: ['assessments']

Loading data from table: 'assessments'...


,id,assessment_timestamp,report_timestamp,timezone,patient_name,patient_number,patient_age,patient_gender,primary_diagnosis,confidence,confidence_percentage,all_diagnoses_json,responses_json,processing_details_json,technical_details_json,clinical_insights_json,created_at
0,MH20251220105009,2025-12-20T10:49:25.299000-05:00,2025-12-20T10:50:09.983872-05:00,America/New_York,Mmmm,Mmmm22,22,Female,Bipolar Type-1,0.862924,86.0,"[{""diagnosis"": ""Bipolar Type-1"", ""probability""...","{""Mood Swing"": ""YES"", ""Sadness"": ""Sometimes"", ...","{""preprocessing_steps"": 3, ""clinical_safety_wa...","{""processing_log"": [""Pipeline_Start: Processin...","{""original_diagnosis"": ""Bipolar Type-1"", ""enha...",2025-12-20 15:50:11.136012


[{"diagnosis": "Bipolar Type-1", "probability": 0.8629241750012676, "confidence_percentage": 86.0, "rank": 1}, {"diagnosis": "Bipolar Type-2", "probability": 0.13148008286924928, "confidence_percentage": 13.0, "rank": 2}, {"diagnosis": "Depression", "probability": 0.005595742129483206, "confidence_percentage": 1.0, "rank": 3}, {"diagnosis": "Normal", "probability": 0.0, "confidence_percentage": 0.0, "rank": 4}]

{"preprocessing_steps": 3, "clinical_safety_warnings": ["Suicidal thoughts detected - please seek professional help immediately"], "total_features_processed": 23, "model_features_used": 21, "feature_engineering_applied": true, "clinical_domains_calculated": true, "clinical_enhancement_applied": false, "safety_check_status": "CRITICAL_ALERTS", "total_diagnoses_considered": 4, "timezone_used": "America/New_York", "assessment_start_time": "2025-12-20T10:49:25.299000-05:00", "report_generation_time": "2025-12-20T10:50:09.983872-05:00", "processing_duration_seconds": 44.684872, "security_validation": "PASSED", "request_language": "en"}

{"processing_log": ["Pipeline_Start: Processing 17 raw features: ['Mood Swing', 'Sadness', 'Euphoric', 'Sleep disorder', 'Exhausted', 'Suicidal thoughts', 'Aggressive Response', 'Nervous Breakdown', 'Overthinking', 'Anorexia', 'Authority Respect', 'Try Explanation', 'Ignore & Move-On', 'Admit Mistakes', 'Concentration', 'Optimism', 'Sexual Activity']", "Safety_Check: Safety warnings: ['Suicidal thoughts detected - please seek professional help immediately']", "Pipeline_Complete: Processed 23 features. Safety OK: False"], "safety_checks_passed": false, "feature_array_shape": [1, 21], "composite_scores_included": true, "probability_distribution": {"min_confidence": 0.0, "max_confidence": 86.29241750012676, "mean_confidence": 25.0, "confidence_range": 86.29241750012676}}

{"original_diagnosis": "Bipolar Type-1", "enhanced_diagnosis": "Bipolar Type-1", "enhancement_applied": false, "adjustment_reasons": [], "pattern_analysis": {"depression_score": 0.4, "bipolar1_score": 0.3333333333333333, "bipolar2_score": 0.6666666666666666, "normal_score": 0.0, "feature_consistency": {"Euphoric": {"value": 1, "expected_min": 2, "expected_max": 3, "consistent": false}, "Mood Swing": {"value": 1, "expected_min": 1, "expected_max": 3, "consistent": true}, "Sleep disorder": {"value": 1, "expected_min": 1, "expected_max": 3, "consistent": true}}, "suggested_adjustments": []}, "confidence_adjustment": 0.0, "original_confidence": 0.8629241750012676}

# delete all the data in the table without deleting the table itself,

In [ ]:
df.columns

# Python Code to Delete the Table

0	MH20251218144802	2025-12-18T14:47:25.521000-05:00	2025-12-18T14:48:02.051283-05:00	America/New_York	Hhhh	Hhhh123	33	Male	Bipolar Type-1	0.83127606	83.0	[{"diagnosis": "Bipolar Type-1", "probability": 0.8312760586893587, "confidence_percentage": 83.0, "rank": 1}, {"diagnosis": "Bipolar Type-2", "probability": 0.15020542279212262, "confidence_percentage": 15.0, "rank": 2}, {"diagnosis": "Depression", "probability": 0.018518518518518517, "confidence_percentage": 2.0, "rank": 3}, {"diagnosis": "Normal", "probability": 0.0, "confidence_percentage": 0.0, "rank": 4}]	{"Mood Swing": "YES", "Sadness": "Sometimes", "Euphoric": "Usually", "Sleep disorder": "Usually", "Exhausted": "Sometimes", "Suicidal thoughts": "YES", "Aggressive Response": "YES", "Nervous Breakdown": "YES", "Overthinking": "YES", "Anorexia": "YES", "Authority Respect": "YES", "Try Explanation": "YES", "Ignore & Move-On": "YES", "Admit Mistakes": "YES", "Concentration": "Poor concentration", "Optimism": "Optimistic", "Sexual Activity": "High interest"}	{"preprocessing_steps": 3, "clinical_safety_warnings": ["Suicidal thoughts detected - please seek professional help immediately", "Aggressive behavior patterns detected - safety assessment recommended", "History of nervous breakdown detected - professional evaluation advised"], "total_features_processed": 23, "model_features_used": 21, "feature_engineering_applied": true, "clinical_domains_calculated": true, "clinical_enhancement_applied": false, "safety_check_status": "CRITICAL_ALERTS", "total_diagnoses_considered": 4, "timezone_used": "America/New_York", "assessment_start_time": "2025-12-18T14:47:25.521000-05:00", "report_generation_time": "2025-12-18T14:48:02.051283-05:00", "processing_duration_seconds": 36.530283, "security_validation": "PASSED", "request_language": "en"}	{"processing_log": ["Pipeline_Start: Processing 17 raw features: ['Mood Swing', 'Sadness', 'Euphoric', 'Sleep disorder', 'Exhausted', 'Suicidal thoughts', 'Aggressive Response', 'Nervous Breakdown', 'Overthinking', 'Anorexia', 'Authority Respect', 'Try Explanation', 'Ignore & Move-On', 'Admit Mistakes', 'Concentration', 'Optimism', 'Sexual Activity']", "Safety_Check: Safety warnings: ['Suicidal thoughts detected - please seek professional help immediately', 'Aggressive behavior patterns detected - safety assessment recommended', 'History of nervous breakdown detected - professional evaluation advised']", "Pipeline_Complete: Processed 23 features. Safety OK: False"], "safety_checks_passed": false, "feature_array_shape": [1, 21], "composite_scores_included": true, "probability_distribution": {"min_confidence": 0.0, "max_confidence": 83.12760586893587, "mean_confidence": 24.999999999999996, "confidence_range": 83.12760586893587}}	{"original_diagnosis": "Bipolar Type-1", "enhanced_diagnosis": "Bipolar Type-1", "enhancement_applied": false, "adjustment_reasons": [], "pattern_analysis": {"depression_score": 0.2, "bipolar1_score": 1.0, "bipolar2_score": 0.6666666666666666, "normal_score": 0.0, "feature_consistency": {"Euphoric": {"value": 2, "expected_min": 2, "expected_max": 3, "consistent": true}, "Mood Swing": {"value": 1, "expected_min": 1, "expected_max": 3, "consistent": true}, "Sleep disorder": {"value": 2, "expected_min": 1, "expected_max": 3, "consistent": true}}, "suggested_adjustments": []}, "confidence_adjustment": 0.0, "original_confidence": 0.8312760586893587}	2025-12-18 19:48:02.434038